## Multipole Moments

Every grid class has the ability to compute the multipole moment integral of a function over cente
Grid can compute various multipole moments of different types:
$$
\begin{align*}
    m_{n_x, n_y, n_z} &= \int (x - X_c)^{n_x} (y - Y_c)^{n_y} (z - Z_c)^{n_z} f(r) dr \qquad \qquad &\text{Cartesian moments}\\
    m_{lm} &= \int | \textbf{r} - \textbf{R}_c|^l S_{l}^m(\theta, \phi) f(\textbf{r}) d\textbf{r} \qquad \qquad &\text{spherical moments} \\
    m_n &= \int | \textbf{r} - \textbf{R}_c|^{n} f(\textbf{r}) d\textbf{r} \qquad \qquad &\text{radial moments}\\
    m_{nlm} &= \int | \textbf{r} - \textbf{R}_c|^{n+1} S_l^m(\theta, \phi) f(\textbf{r}) d\textbf{r} \qquad \qquad &\text{radial with spherical moments}
\end{align*}
$$
for some function $f : \mathbb{R}^3\rightarrow \mathbb{R}$, where $S_l^m$ is the regular, real solid harmonics, $(n_x, n_y, n_z)$ are the Cartesian orders over some center $(X_c, Y_c, Z_c)$ and $(l, m)$ are the angular order and degree.


This example illustrates how to compute the dipole moment of Formaldehyde.



IOData is used to first read the wavefunction information of Formaldehyde.

In [36]:
from iodata import load_one

mol = load_one("./ch2o_q+0.fchk")

print("Dipole Moments ", mol.moments)

Dipole Moments  {(1, 'c'): array([-9.39793529e-01,  2.44832724e-08, -2.02253053e-07])}


In order to compute the moment integral, a molecular grid class is constructed.

In [37]:
from grid.becke import BeckeWeights
from grid.molgrid import MolGrid
from grid.onedgrid import GaussLegendre
from grid.rtransform import BeckeRTransform

# Construct a radial grid
oned_grid = GaussLegendre(npoints=50)
radial_grid = BeckeRTransform(0.0, R=1.5).transform_1d_grid(oned_grid)  #BeckeRTransform(0.0, R=2.0).transform_1d_grid(oned_grid)

# Construct Molecular grid with angular degree of 50 for each atom.
mol_grid = MolGrid.from_size(
    atnums=mol.atnums,          # The atomic numbers of Formaldehyde
    atcoords=mol.atcoords,      # The atomic coordinates of Formaldehyde
    rgrid=radial_grid,          # Radial grid used to construct atomic grids over each carbon, and hydrogen.
    size=50,                    # The angular degree of the atomic grid over each carbon, and hydrogen.
    aim_weights=BeckeWeights(), # Atom-in molecular weights: Becke weights
)

The moment integrals can be calculated.

In [55]:
import numpy as np
from gbasis.wrappers import from_iodata
from gbasis.evals.density import evaluate_density

# Construct molecular basis from wave-function information read by IOData
basis, type = from_iodata(mol)

# Compute the electron density
rdm = mol.one_rdms["scf"]
electron_density = evaluate_density(rdm, basis, mol_grid.points, coord_type=type)


center_atom = np.array([np.sum(mol.atcoords, axis=0) / len(mol.atcoords)])
a = mol_grid.moments(
    3, np.array([[0.0, 0.0, 0.0]]), electron_density, type_mom="cartesian", return_orders=True
)
print(a[0][-6])
print(np.sum(mol.atnums[:, np.newaxis] * mol.atcoords, axis=0))
print(a[0][-6] + np.sum(mol.atnums[:, np.newaxis] * mol.atcoords, axis=0))
# (2, 'c'): array([ 1.24344979e-14, -4.79884555e-15,  4.53864177e-15, -1.77635684e-15, 3.25806694e-15, -5.32907052e-15])}
# Dipole Moments  {(1, 'c'): array([-9.39793529e-01,  2.44832724e-08, -2.02253053e-07])}

[-0.00395472]
[ 1.60950630e+01  3.45000845e-06 -1.31242995e-06]
[ 1.60911083e+01 -3.95126670e-03 -3.95602914e-03]
